# Importing Dependencies

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xg
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Getting Data Information

In [ ]:
dataset = pd.read_csv('/kaggle/input/nba2k20-player-dataset/nba2k-full.csv')

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

# Handling Attributes

In [ ]:
dataset["team"].unique()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset = dataset.drop(['full_name','b_day','weight','height','college','version'],axis=1)
dataset.head()

In [ ]:
dataset=dataset.dropna()
dataset.isnull().sum()

In [ ]:
dataset.shape

# Feature Engineering

In [ ]:
def remove(value):
    value=value[1:]
    return int(value)

dataset['jersey'] = dataset['jersey'].apply(remove)

In [ ]:
dataset['salary'] = dataset['salary'].apply(remove)
dataset.head()

In [ ]:
dataset['country'].value_counts()

In [ ]:
def others(value):
    if value not in ['USA', 'Canada', 'Australia']:
        return 'Others'
    else:
        return value
dataset['country'] = dataset['country'].apply(others)

In [ ]:
dataset['country'].value_counts()

In [ ]:
dataset['position'].value_counts()

In [ ]:
def removeundraft(value):
    if value=='Undrafted':
        return
    else:
        return int(value)

In [ ]:
dataset['draft_round'] = dataset['draft_round'].apply(removeundraft)
dataset['draft_peak'] = dataset['draft_peak'].apply(removeundraft)

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset = dataset.dropna()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.shape

# Analysing Correlation

In [ ]:
dataset.corr()

In [ ]:
sns.heatmap(dataset.corr())

# Data Visualising and Analysing

In [ ]:
plt.figure(figsize=(8,6))
sns.pairplot(dataset)
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 6))
sns.boxplot(dataset['salary'], ax=axes[0])
sns.boxplot(dataset['rating'], ax=axes[1])
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.jointplot(dataset, x='rating', y='salary', kind='reg')

In [ ]:
sns.scatterplot(dataset['salary'])

# Label Encoding

In [ ]:
le=LabelEncoder()
dataset['position']=le.fit_transform(dataset['position'])
dataset['country']=le.fit_transform(dataset['country'])
dataset['team']=le.fit_transform(dataset['team'])
dataset.head()

# Dividing Target and Attributes

In [ ]:
X = dataset.drop(['salary'],axis=1)
Y = dataset['salary']

# Using Train Test Split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
Y_train.shape

# Model Building and Predictions

In [ ]:
model = {
 'Random Forest Regressor' : RandomForestRegressor(),
 'Gradient Boost Regressor' : GradientBoostingRegressor(),
 'XGBoost' : xg.XGBRegressor(),
 'XGRF Regressor' : xg.XGBRFRegressor(),
 'Support Vector regressor' : SVR(),
 'Lasso Reg' : Lasso(),
 'Ridge Reg' : Ridge(),
 'Linear Regression' : LinearRegression(),
 'Extra Trees Regressor' : ExtraTreesRegressor()
}

In [ ]:
pred ={}
for name, model in model.items():
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    pred[name]=y_pred

# Analysing Predictions

In [ ]:
acc= {} 
for name, y_pred in pred.items():
    mse = mean_squared_error(Y_test, y_pred)
    mae = mean_absolute_error(Y_test, y_pred)
    r2 = r2_score(Y_test, y_pred)
    acc[name] = r2
    print(f"Results for {name} : ")
    print (f"Mean Square Error : {mse}")
    print (f"Mean Absolute Error : {mae}")
    print(f"R2 Score : {r2}")
    plt.figure(figsize=(15, 6))

# Plot Actual vs. Predicted values
    plt.subplot(1, 2, 1)
    plt.plot(np.arange(len(Y_test)), Y_test, label='Actual Trend')
    plt.plot(np.arange(len(Y_test)), y_pred, label='Predicted Trend')
    plt.xlabel('Data')
    plt.ylabel('Trend')
    plt.legend()
    plt.title('Actual vs. Predicted')

    # Plot Residuals
    residuals = Y_test - y_pred

    plt.subplot(1, 2, 2)
    plt.hist(residuals)
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.title('Residual Plot')

    plt.tight_layout()
    plt.show()